In [66]:
from tkinter import *
from tkinter.font import Font
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk , ImageEnhance
import cv2 as cv
import numpy as np
import cv2
import imutils
import easyocr

In [68]:
class Application(Tk):
    img = None
    img_is_found = False
    ifile = '' 
    
    
        
    def window_size(self):
        width = 1500 # Width 
        height = 650 # Height
        screen_width = self.winfo_screenwidth()  # Width of the screen
        screen_height = self.winfo_screenheight() # Height of the screen
        # Calculate Starting X and Y coordinates for Window
        x = (screen_width/2) - (width/2)
        y = (screen_height/2) - (height/2)
        self.geometry('%dx%d+%d+%d' % (width, height, x, y))
    
    def menuBar_edit(self):
        ## 1
        font_edit = Font(size = 13)
        self.menu1 = Menubutton(self , text='File',activebackground='#e9ecef',activeforeground = '#0096c7' ,font = Font(size = 15,weight='bold'),bg = '#ced4da',fg = '#0D47A1')
        self.menu1.grid(row = 0,column=0,padx = 4)
        self.menu1.menu = Menu(self.menu1, tearoff=0)
        self.menu1["menu"] = self.menu1.menu

        self.menu1.menu.add_command(label='Open',command= self.choose,font =font_edit  )
        self.menu1.menu.add_command(label="Save",command = self.savefile,font =font_edit)
        self.menu1.menu.add_command(label="Save as",command = self.save_as_file,font =font_edit)
        self.menu1.menu.add_command(label='Exit',command = self.destroy,font =font_edit)
        ## 2
        self.menu4 = Menubutton(self, text='Image OCR',activebackground='#495057',activeforeground = '#F5F3F4',font = Font(size = 15,weight='bold'),width = 20,bg = '#ced4da',fg = '#0D47A1')
        self.menu4.grid(row=0,column=1)
        self.menu4.menu = Menu(self.menu4, tearoff=0)
        self.menu4["menu"] = self.menu4.menu
        self.menu4.menu.add_command(label = 'OCR',font =font_edit,command = self.ocr)

        
        
    #########################################################################
    #########################################################################    
    #########################################################################
    #########################################################################
    
        
    
        # =============================================
    def choose(self):
        self.ifile=filedialog.askopenfilename(parent=self, title='Choose a file')
        if self.ifile:
            path = Image.open(self.ifile)
            self.path_edit = path
            # Resize Image
            self.image2=path.resize((600,400))
            # ====================================
            self.image2 = ImageTk.PhotoImage(self.image2)
            self.label.configure(image=self.image2)
            self.label.image = self.image2
            self.img = np.array(path)
            self.img_is_found = True
            self.ifile = self.ifile
            self.Img_original.configure(text='Original Image')
            self.label.place(x  = 450 , y = 90)
            self.Img_original.place(x  = 685 , y = 555)
            self.label3.place_forget()
            self.Image_Edit.place_forget()
            #====
            self.label2.place_forget()
            self.label3.place_forget()
            self.Restoration_Filter.place_forget()
            
            
            
    def savefile(self):
        edge = self.img_save
        edge.save('new.jpg')
        messagebox.showinfo('Alert','Image Save')
        
        
    
    def save_as_file(self):
        filename = filedialog.asksaveasfile(mode='w', defaultextension=".jpg")
        if not filename:
            return
        edge = self.img_save
        edge.save(filename)
        messagebox.showinfo('Alert','Image Save')
        
    

     # =============================================
    def call_Widget(self):
        self.label.place(x  = 70 , y = 100)
        self.Img_original.place(x  = 300 , y = 500)
        # ======== label Page 
        self.label_page.place(relx=0.2, rely=0.05)
        # =================
        self.label2.place(x  = 840 , y = 100)
        self.label3.place(x  = 840 , y = 550)
        self.Restoration_Filter.place(x  = 1050 , y = 500)
          
    # ==============================================================
    # ==============================================================
    # ==============================================================
    # ==============================================================

        
    # **************************************************************
    def ocr(self):
        path = self.ifile
        self.img_original = cv.imread(path , 1)
        self.img_original = cv.cvtColor(self.img_original, cv.COLOR_BGR2RGB)
        gray = cv2.cvtColor(self.img_original,cv2.COLOR_BGR2GRAY)
        bfilter = cv2.bilateralFilter(gray, 11, 17, 17) #Noise reduction
        edged = cv2.Canny(bfilter, 30, 200) #Edge detection
        keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contourss = imutils.grab_contours(keypoints)
        contours = sorted(contourss, key=cv2.contourArea, reverse=True)[:10]
        location = None
        for contour in contours:
            approx = cv2.approxPolyDP(contour, 10, True)
            if len(approx) == 4:
                location = approx
                break
        mask = np.zeros(gray.shape, np.uint8)
        new_image = cv2.drawContours(mask, [location], -1,255, -1)
        new_image = cv2.bitwise_and(self.img_original, self.img_original, mask=mask)
        (x,y) = np.where(mask==255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        cropped_image = gray[x1:x2+1, y1:y2+1]  
        ######
        # Resize Image
        self.img_save_1 = Image.fromarray(cropped_image)
        # Resize Image
        img_after_new=self.img_save_1.resize((600,200))            
        # ====================================
        img_after_2 = ImageTk.PhotoImage(img_after_new)
        ######
        reader = easyocr.Reader(['en'])
        result = reader.readtext(cropped_image)
        text = result[0][-2]
        font = cv2.FONT_HERSHEY_SIMPLEX
        res = cv2.putText(self.img_original, text=text, org=(approx[0][0][0], approx[1][0][1]+60), fontFace=font, fontScale=1, color=(0,255,0), thickness=2, lineType=cv2.LINE_AA)
        res = cv2.rectangle(self.img_original, tuple(approx[0][0]), tuple(approx[2][0]), (0,255,0),3)
        # Resize Image
        self.img_save = Image.fromarray(res)
        # Resize Image
        img_after_new=self.img_save.resize((600,400))            
        # ====================================
        img_after = ImageTk.PhotoImage(img_after_new)
            
        self.label2.configure(image = img_after)
        self.label2.image = img_after
        
        self.label3.configure(image = img_after_2)
        self.label3.image = img_after_2
        
        self.call_Widget()
                    
        #====            
        self.Restoration_Filter.configure(text='Plat Number')
        self.Restoration_Filter.text = 'Plat Number'
            
        self.label_page.configure(text='Optical Character Recognition')  
    #===============================================
    def __init__(self):
        Tk.__init__(self)
#         self.attributes('-fullscreen', True)
        self.state('zoomed')
        self.title('Digital Image Processing')
        self.config(bg = '#e9ecef')

        self.sideframe  = Frame(self, bg = '#CED4DA', width = 1600 ,  height = 35)
        
        # For Noise
        
        self.sideframe.place(x  = 0 , y = 0)
        self.menuBar_edit()
        # ======== label Page
        self.label_page = Label(self, text=None ,font = Font(size = 30 ,weight='bold'),width = 40,justify = CENTER,fg = '#0D47A1',bg = '#e9ecef')
        # ========
        self.Img_original = Label(self, text=None,font = Font(size = 15 ,weight='bold'),bg = '#e9ecef',fg = '#0D47A1')
        self.Restoration_Filter = Label(self  ,font = Font(size = 15 ,weight='bold'),bg = '#e9ecef',fg = '#0D47A1')        
        self.Image_Edit = Label(self ,text=None ,font = Font(size = 15 ,weight='bold'),bg = '#e9ecef',fg = '#0D47A1')
        
        self.label  = Label(image=None, width=600, height=400, bg = '#e9ecef')
        self.label2 = Label(image=None, width=600, height=400, bg = '#e9ecef')
        self.label3 = Label(image=None, width=600, height=200, bg = '#e9ecef')
        
        
if __name__ == "__main__":
    ui = Application()
    ui.mainloop()

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
